In [ ]:
!pip install -U pandasql -q

  Preparing metadata (setup.py) ... done


In [ ]:
!pip install pyforest -q

  Preparing metadata (setup.py) ... done


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from pandasql import sqldf
import pyforest

In [ ]:
%cd /content/drive/MyDrive/UTS/SPR23/iLab/Dataset/by_wistleout

/content/drive/MyDrive/UTS/SPR23/iLab/Dataset/by_wistleout


In [ ]:
ls

DimUTSCampaign.csv     DimUTSMobileProduct.csv   DimUTSSupplier.csv
DimUTSIspProduct.csv   DimUTSProduct.csv         DimUTSURL.csv
DimUTSMobilePhone.csv  DimUTSSearchCriteria.csv  FactUTSPageImpression.csv


# Fact Table

In [ ]:
fact_df = pd.read_csv('/content/drive/Shareddrives/protik/iLab/Dataset/data/FactUTSPageImpression.csv')

<ipython-input-7-5c7884fda150>:1: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  fact_df = pd.read_csv('/content/drive/Shareddrives/protik/iLab/Dataset/data/FactUTSPageImpression.csv')


In [ ]:
fact_df.head(5)

,ID,ViewedOn,ViewedOnDate,ProductKey,CampaignKey,SearchCriteriaKey,MobilePhoneKey,UserIdKey,URLKey,LandingURLKey,...,WOHeadlineExtraDataDiscount,OneOffFeeTotal,MonthlyFeeTotal,AnnualFeeTotal,Phoneupfrontcost,Phonemonthlycost,Phonefinancingterm,PhoneEarlyupgradeterm,Phoneleaseterm,Phonefinancetypedesc
0,276281191,2022-12-01 12:09:56.7970000,2022-12-01 00:00:00.0000000,0,0,1283,0,30763830,218641,218641,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1,278966940,2023-01-28 10:12:36.3770000,2023-01-28 00:00:00.0000000,0,0,1283,0,43749662,40104,40104,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,279637545,2023-02-10 15:48:24.4800000,2023-02-10 00:00:00.0000000,0,0,1283,0,44239405,62097,45538,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
3,277464744,2022-12-29 10:21:00.7200000,2022-12-29 00:00:00.0000000,0,0,1283,0,32428604,40104,40104,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
4,279014393,2023-01-29 11:32:02.9200000,2023-01-29 00:00:00.0000000,0,0,1283,0,31285621,32641,47995,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN


In [ ]:
fact_df['ViewedOnDate'] = pd.to_datetime(fact_df['ViewedOnDate'])
fact_df['ViewedOn'] = pd.to_datetime(fact_df['ViewedOn'])

# Extract the time part and update the "ViewedOn" column
fact_df['ViewedOn'] = fact_df['ViewedOn'].dt.strftime('%H:%M:%S')

In [ ]:
fact_df.rename(columns={'ViewedOn': 'ViewedOnTime'}, inplace=True)

In [ ]:
fact_df.shape

(1167991, 40)

In [ ]:
fact_df = fact_df[fact_df['ViewedOnDate']>='2022-12-01']

In [ ]:
fact_df.shape

(1115188, 40)

In [ ]:
fact_df = fact_df[fact_df['ProductAreaName'] == 'Mobile Phones']

In [ ]:
fact_df.shape

(483275, 40)

In [ ]:
fact_df.columns

Index(['ID', 'ViewedOnTime', 'ViewedOnDate', 'ProductKey', 'CampaignKey',
       'SearchCriteriaKey', 'MobilePhoneKey', 'UserIdKey', 'URLKey',
       'LandingURLKey', 'ReferrerURLKey', 'FirstReferrerURLKey', 'SupplierKey',
       'PrevPageURLKey', 'TransactionCount', 'PageCount', 'IsAd',
       'IsTransaction', 'IsProduct', 'PrevID', 'AffiliateName',
       'DeviceTypeDesc', 'ContractLengthMonths', 'ContractLengthDesc',
       'ProductAreaName', 'Source', 'Medium', 'Campaign',
       'WOHeadlinePlanFixedDiscount', 'WOHeadlinePhoneDiscount',
       'WOHeadlineExtraDataDiscount', 'OneOffFeeTotal', 'MonthlyFeeTotal',
       'AnnualFeeTotal', 'Phoneupfrontcost', 'Phonemonthlycost',
       'Phonefinancingterm', 'PhoneEarlyupgradeterm', 'Phoneleaseterm',
       'Phonefinancetypedesc'],
      dtype='object')

In [ ]:
fact_df[fact_df['IsTransaction'] == 1]['SearchCriteriaKey'].unique()

array([0])

In [ ]:
fact_df.drop(columns=['SearchCriteriaKey',
                         'ProductAreaName',
                         'Phonefinancingterm',
                         'PhoneEarlyupgradeterm',
                         'Phoneleaseterm',
                         'Phonefinancetypedesc',
                         'MobilePhoneKey',
                         'URLKey',
                         'LandingURLKey',
                         'ReferrerURLKey',
                         'FirstReferrerURLKey',
                         'PrevPageURLKey',
                         'PrevID'], inplace=True)

fact_df.head()

,ID,ViewedOnTime,ViewedOnDate,ProductKey,CampaignKey,UserIdKey,SupplierKey,TransactionCount,PageCount,IsAd,...,Medium,Campaign,WOHeadlinePlanFixedDiscount,WOHeadlinePhoneDiscount,WOHeadlineExtraDataDiscount,OneOffFeeTotal,MonthlyFeeTotal,AnnualFeeTotal,Phoneupfrontcost,Phonemonthlycost
2,279637545,15:48:24,2023-02-10,0,0,44239405,0,0,1,0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,278759186,23:12:18,2023-01-23,0,0,29305574,0,0,1,0,...,cpc,0123_MOB_7_Optus_Hot_Deals,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,279715670,12:20:27,2023-02-12,0,0,45261797,0,0,1,0,...,cpc,[PS-MOB] Sim,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,277589212,07:07:54,2023-01-01,0,0,44590967,0,0,1,0,...,cpc,[PS-MOB] Mobile Plans,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,277141899,14:04:45,2022-12-21,0,0,30817233,0,0,1,0,...,cpc,1222_MOB_5_Pixel7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
fact_df.columns

Index(['ID', 'ViewedOnTime', 'ViewedOnDate', 'ProductKey', 'CampaignKey',
       'UserIdKey', 'SupplierKey', 'TransactionCount', 'PageCount', 'IsAd',
       'IsTransaction', 'IsProduct', 'AffiliateName', 'DeviceTypeDesc',
       'ContractLengthMonths', 'ContractLengthDesc', 'Source', 'Medium',
       'Campaign', 'WOHeadlinePlanFixedDiscount', 'WOHeadlinePhoneDiscount',
       'WOHeadlineExtraDataDiscount', 'OneOffFeeTotal', 'MonthlyFeeTotal',
       'AnnualFeeTotal', 'Phoneupfrontcost', 'Phonemonthlycost'],
      dtype='object')

## Dropping duplicates

In [ ]:
fact_df = fact_df.drop_duplicates()

In [ ]:
fact_df = fact_df.drop_duplicates()
fact_df.isnull().any()

ID                             False
ViewedOnTime                   False
ViewedOnDate                   False
ProductKey                     False
CampaignKey                    False
UserIdKey                      False
SupplierKey                    False
TransactionCount               False
PageCount                      False
IsAd                           False
IsTransaction                  False
IsProduct                      False
AffiliateName                  False
DeviceTypeDesc                 False
ContractLengthMonths           False
ContractLengthDesc             False
Source                          True
Medium                          True
Campaign                        True
WOHeadlinePlanFixedDiscount    False
WOHeadlinePhoneDiscount        False
WOHeadlineExtraDataDiscount     True
OneOffFeeTotal                 False
MonthlyFeeTotal                False
AnnualFeeTotal                 False
Phoneupfrontcost               False
Phonemonthlycost               False
d

## Missing values

In [ ]:
# Calculate the total number of missing values per column
missing_values = fact_df.isnull().sum()

# Calculate the percentage of missing values per column
percentage_missing = (missing_values / len(fact_df)) * 100

# Create a DataFrame to display the results
missing_data_info = pd.DataFrame({
    'Column Name': missing_values.index,
    'Total Missing Values': missing_values.values,
    'Percentage Missing': percentage_missing.values
})

# Sort the DataFrame by percentage of missing values (descending)
missing_data_info = missing_data_info.sort_values(by='Percentage Missing', ascending=False)

# Print the missing data information
missing_data_info

,Column Name,Total Missing Values,Percentage Missing
18,Campaign,136529,28.250789
17,Medium,136529,28.250789
16,Source,136504,28.245616
21,WOHeadlineExtraDataDiscount,456,0.094356
0,ID,0,0.000000
14,ContractLengthMonths,0,0.000000
25,Phoneupfrontcost,0,0.000000
24,AnnualFeeTotal,0,0.000000
23,MonthlyFeeTotal,0,0.000000
22,OneOffFeeTotal,0,0.000000


In [ ]:
# Filter for columns within 0 to 50% missing values
columns_within_0_to_50_percent_missing = missing_data_info[(missing_data_info['Percentage Missing'] > 0) & (missing_data_info['Percentage Missing'] <= 50)]

# Print the filtered information
print("Columns within 0 to 50% missing values:")
print(columns_within_0_to_50_percent_missing)

Columns within 0 to 50% missing values:
                    Column Name  Total Missing Values  Percentage Missing
18                     Campaign                136529           28.250789
17                       Medium                136529           28.250789
16                       Source                136504           28.245616
21  WOHeadlineExtraDataDiscount                   456            0.094356


In [ ]:
# Extract the column names from the filtered DataFrame
columns_to_fill_with_other_values = columns_within_0_to_50_percent_missing['Column Name'].tolist()

In [ ]:
# Update specific columns
for column in columns_to_fill_with_other_values:
    if column in ['Medium', 'Campaign', 'Source']:
        fact_df[column] = fact_df[column].fillna('unknown')
    elif column == 'WOHeadlineExtraDataDiscount':
        fact_df[column] = fact_df[column].fillna(0)

In [ ]:
fact_df.columns

Index(['ID', 'ViewedOnTime', 'ViewedOnDate', 'ProductKey', 'CampaignKey',
       'UserIdKey', 'SupplierKey', 'TransactionCount', 'PageCount', 'IsAd',
       'IsTransaction', 'IsProduct', 'AffiliateName', 'DeviceTypeDesc',
       'ContractLengthMonths', 'ContractLengthDesc', 'Source', 'Medium',
       'Campaign', 'WOHeadlinePlanFixedDiscount', 'WOHeadlinePhoneDiscount',
       'WOHeadlineExtraDataDiscount', 'OneOffFeeTotal', 'MonthlyFeeTotal',
       'AnnualFeeTotal', 'Phoneupfrontcost', 'Phonemonthlycost'],
      dtype='object')

In [ ]:
constant_columns = []

for column in fact_df.columns:
    if fact_df[column].nunique() == 1:
        constant_columns.append(column)

# Print the constant columns
print("Constant Columns:")
print(constant_columns)

Constant Columns:
['MonthlyFeeTotal', 'AnnualFeeTotal', 'Phoneupfrontcost']


In [ ]:
fact_df = fact_df.drop(columns=constant_columns)

In [ ]:
fact_df.columns

Index(['ID', 'ViewedOnTime', 'ViewedOnDate', 'ProductKey', 'CampaignKey',
       'UserIdKey', 'SupplierKey', 'TransactionCount', 'PageCount', 'IsAd',
       'IsTransaction', 'IsProduct', 'AffiliateName', 'DeviceTypeDesc',
       'ContractLengthMonths', 'ContractLengthDesc', 'Source', 'Medium',
       'Campaign', 'WOHeadlinePlanFixedDiscount', 'WOHeadlinePhoneDiscount',
       'WOHeadlineExtraDataDiscount', 'OneOffFeeTotal', 'Phonemonthlycost'],
      dtype='object')

In [ ]:
fact_df.rename(columns={'ID':'session_ID'}, inplace=True)

In [ ]:
fact_df.columns

Index(['session_ID', 'ViewedOnTime', 'ViewedOnDate', 'ProductKey',
       'CampaignKey', 'UserIdKey', 'SupplierKey', 'TransactionCount',
       'PageCount', 'IsAd', 'IsTransaction', 'IsProduct', 'AffiliateName',
       'DeviceTypeDesc', 'ContractLengthMonths', 'ContractLengthDesc',
       'Source', 'Medium', 'Campaign', 'WOHeadlinePlanFixedDiscount',
       'WOHeadlinePhoneDiscount', 'WOHeadlineExtraDataDiscount',
       'OneOffFeeTotal', 'Phonemonthlycost'],
      dtype='object')

## Unnecesary Columns

In [ ]:
columns_to_drop=[
    'TransactionCount' # same value as IsTransaction
]
fact_df = fact_df.drop(columns=columns_to_drop)

## Renaming columns for better understanding

In [ ]:
fact_df.rename(columns={'DeviceTypeDesc': 'DeviceType'}, inplace=True)

In [ ]:
fact_df.shape

(483275, 23)

In [ ]:
print(fact_df['Source'].unique())

['unknown' 'facebook' 'google' 'facebook_OG_WO' 'bing'
 'Widget_MobilePhoneSearch' 'CampaignMonitor' 'facebook_SC_WO'
 'Widget_BroadbandSearch' 'Widget_PhoneFinder'
 'Widget_MobilePhoneProductCta,google' 'pinterest_OG_SW'
 'Widget_MobilePhoneSearch,google' 'Widget_MobilePhoneProductCta'
 'Widget_MobilePhoneSearch,bing' 'twitter_OG_WO'
 'Widget_BroadbandProductCta' 'Widget_MobilePhoneProductCompact'
 'Widget_MobilePhonePopularSuppliers' 'Widget_MobilePhonePopularPlans'
 'facebook_OR_WO' 'pocket_saves' 'Widget_MobilePhoneProduct'
 'Widget_BroadbandProductCta,google'
 'Widget_MobilePhoneProductCta,google,google'
 'Widget_MobilePhoneProductCta,bing' 'Widget_Phone'
 'Widget_MobilePhoneProductCompact,bing'
 'Widget_MobilePhoneProduct,Widget_MobilePhoneProduct'
 'wo,Widget_Fairfax_SMH_DigitalLife' 'newsletter'
 'Widget_BroadbandSearchForm' 'Widget_BroadbandSearch,bing'
 'Widget_MobilePhoneProductCta,CampaignMonitor'
 'Widget_MobilePhoneProductCompact,bing,bing'
 'Widget_MobilePhoneProductComp

In [ ]:
print(fact_df['Medium'].unique())

['unknown' 'cpc' 'social' 'Facebook' 'Widget' 'email' 'Widget,cpc'
 'Pinterest' 'Twitter' 'twitter' 'Widget,Facebook' 'Widget,cpc,cpc'
 'Widget,Facebook,Facebook' 'Widget,Widget' 'affil,Widget']


In [ ]:
fact_df['CampaignKey'].nunique()

617

In [ ]:
# facebook inside cpc, separte name as facebook too
# what is widget cpc
#

# DimUTSProduct Table

In [ ]:
ls

DimUTSCampaign.csv     DimUTSMobileProduct.csv   DimUTSSupplier.csv
DimUTSIspProduct.csv   DimUTSProduct.csv         DimUTSURL.csv
DimUTSMobilePhone.csv  DimUTSSearchCriteria.csv  FactUTSPageImpression.csv


In [ ]:
product_df = pd.read_csv('/content/drive/Shareddrives/protik/iLab/Dataset/data/DimUTSProduct.csv')

In [ ]:
product_df.head(5)

,ProductKey,ID,StartDate,EndDate,Name,CurrentName,Active,Preview,Deleted,BaseCost,...,PaymentFrequencyDerivedOrder,PaymentPeriod,CustomerTypeName,SupplierID,ProductAreaName,BonusValueDescription,GlobalShowOnWhistleout,IspProductKey,MobileProductKey,EditorsPickName
0,12597,12984,2023-01-27 00:00:00.0000000,2023-01-31 00:00:00.0000000,100GB Data SIM Only,100GB Data SIM Only,1,0.0,0,35.0,...,31.0,1.0,Personal,515,Broadband,Download speeds capped at 100 Mbps on 4G and 5...,1.0,6586,0,Best Mobile Broadband - High Use
1,12509,12984,2023-02-01 00:00:00.0000000,2023-02-17 00:00:00.0000000,100GB Data SIM Only,100GB Data SIM Only,1,0.0,0,35.0,...,31.0,1.0,Personal,515,Broadband,Download speeds capped at 100 Mbps on 4G and 5...,1.0,6586,0,Best Mobile Broadband - High Use
2,14680,12984,2023-02-21 00:00:00.0000000,2023-02-28 00:00:00.0000000,100GB Data SIM Only,100GB Data SIM Only,1,0.0,0,35.0,...,31.0,1.0,Personal,515,Broadband,Download speeds capped at 100 Mbps on 4G and 5...,1.0,6586,0,Best Mobile Broadband - High Use
3,10632,12994,2023-01-24 00:00:00.0000000,2023-02-07 00:00:00.0000000,4G Home Internet Unlimited (M2M),4G Home Internet Unlimited (M2M),1,0.0,0,59.0,...,31.0,1.0,Personal,254,Broadband,"$216 upfront cost for 4G WiFi Modem - unbox, p...",1.0,4309,0,NaN
4,13500,12994,2023-02-16 00:00:00.0000000,2023-02-28 00:00:00.0000000,4G Home Internet Unlimited (M2M),4G Home Internet Unlimited (M2M),1,1.0,0,59.0,...,31.0,1.0,Personal,254,Broadband,"$216 upfront cost for 4G WiFi Modem - unbox, p...",0.0,4309,0,NaN


In [ ]:
product_df.shape

(1599, 23)

In [ ]:
product_df.columns

Index(['ProductKey', 'ID', 'StartDate', 'EndDate', 'Name', 'CurrentName',
       'Active', 'Preview', 'Deleted', 'BaseCost', 'PaymentFrequencyID',
       'PaymentFrequency', 'PaymentFrequencyDerived',
       'PaymentFrequencyDerivedOrder', 'PaymentPeriod', 'CustomerTypeName',
       'SupplierID', 'ProductAreaName', 'BonusValueDescription',
       'GlobalShowOnWhistleout', 'IspProductKey', 'MobileProductKey',
       'EditorsPickName'],
      dtype='object')

In [ ]:
product_df = product_df[product_df['ProductAreaName'] == 'Mobile Phones']

In [ ]:
product_df.shape

(701, 23)

In [ ]:
product_df = product_df[product_df['EndDate']>='2022-12-01']

In [ ]:
product_df.shape

(659, 23)

In [ ]:
product_df = product_df.drop_duplicates()
product_df.isnull().any()

ProductKey                      False
ID                              False
StartDate                       False
EndDate                         False
Name                            False
CurrentName                     False
Active                          False
Preview                         False
Deleted                         False
BaseCost                        False
PaymentFrequencyID              False
PaymentFrequency                False
PaymentFrequencyDerived         False
PaymentFrequencyDerivedOrder    False
PaymentPeriod                   False
CustomerTypeName                False
SupplierID                      False
ProductAreaName                 False
BonusValueDescription            True
GlobalShowOnWhistleout          False
IspProductKey                   False
MobileProductKey                False
EditorsPickName                  True
dtype: bool

In [ ]:
# Calculate the total number of missing values per column
missing_values = product_df.isnull().sum()

# Calculate the percentage of missing values per column
percentage_missing = (missing_values / len(product_df)) * 100

# Create a DataFrame to display the results
missing_data_info = pd.DataFrame({
    'Column Name': missing_values.index,
    'Total Missing Values': missing_values.values,
    'Percentage Missing': percentage_missing.values
})

# Sort the DataFrame by percentage of missing values (descending)
missing_data_info = missing_data_info.sort_values(by='Percentage Missing', ascending=False)

# Print the missing data information
missing_data_info

,Column Name,Total Missing Values,Percentage Missing
22,EditorsPickName,630,95.599393
18,BonusValueDescription,7,1.062215
12,PaymentFrequencyDerived,0,0.000000
21,MobileProductKey,0,0.000000
20,IspProductKey,0,0.000000
19,GlobalShowOnWhistleout,0,0.000000
17,ProductAreaName,0,0.000000
16,SupplierID,0,0.000000
15,CustomerTypeName,0,0.000000
14,PaymentPeriod,0,0.000000


In [ ]:
# Filter for columns with more than 50% missing values
columns_with_more_than_50_percent_missing = missing_data_info[missing_data_info['Percentage Missing'] > 50]

# Print the filtered information
print("Columns with more than 50% missing values:")
print(columns_with_more_than_50_percent_missing)

Columns with more than 50% missing values:
        Column Name  Total Missing Values  Percentage Missing
22  EditorsPickName                   630           95.599393


In [ ]:
# Fill missing values in the 'EditorsPickName' column with 0
product_df['EditorsPickName'] = product_df['EditorsPickName'].fillna(0)

# Replace non-zero values with 1
product_df['EditorsPickName'] = np.where(product_df['EditorsPickName'] != 0, 1, 0)

<IPython.core.display.Javascript object>

In [ ]:
product_df.EditorsPickName.unique()

array([0, 1])

In [ ]:
product_df.columns

Index(['ProductKey', 'ID', 'StartDate', 'EndDate', 'Name', 'CurrentName',
       'Active', 'Preview', 'Deleted', 'BaseCost', 'PaymentFrequencyID',
       'PaymentFrequency', 'PaymentFrequencyDerived',
       'PaymentFrequencyDerivedOrder', 'PaymentPeriod', 'CustomerTypeName',
       'SupplierID', 'ProductAreaName', 'BonusValueDescription',
       'GlobalShowOnWhistleout', 'IspProductKey', 'MobileProductKey',
       'EditorsPickName'],
      dtype='object')

In [ ]:
# Define the substrings to search for in column names
substrings_to_search = ['key', 'comment', 'id']

# Get the column names that contain those word
matching_columns = [col for col in product_df.columns if any(sub in col.lower() for sub in substrings_to_search)]

# Print the matching column names
print("Columns containing 'key,' 'comment,' or 'id' in their names:")
for col in matching_columns:
    print(col)

Columns containing 'key,' 'comment,' or 'id' in their names:
ProductKey
ID
PaymentFrequencyID
SupplierID
IspProductKey
MobileProductKey


In [ ]:
'''
ProductKey = unique key for the product type
ID = unique identifier of an individual product
'''
columns_to_drop = [
    'ProductAreaName',
    'IspProductKey',
    'PaymentFrequencyID',
    'BonusValueDescription',
    'PaymentFrequencyDerived' # same as PaymentFrequencyDerivedOrder which is presented with days

]
# Drop the specified columns
product_df = product_df.drop(columns=columns_to_drop)

In [ ]:
product_df.shape

(659, 18)

In [ ]:
product_df.rename(columns={'PaymentFrequencyDerivedOrder': 'PaymentFrequencyDerived_in_days', 'ID':'product_indv_ID'}, inplace=True)

In [ ]:
constant_columns = []

for column in product_df.columns:
    if product_df[column].nunique() == 1:
        constant_columns.append(column)

# Print the constant columns
print("Constant Columns:")
print(constant_columns)

Constant Columns:
['Deleted', 'CustomerTypeName']


In [ ]:
columns_to_drop = [
    'Deleted',
    'CustomerTypeName'
]
# Drop the specified columns
product_df = product_df.drop(columns=columns_to_drop)

In [ ]:
product_df.columns

Index(['ProductKey', 'product_indv_ID', 'StartDate', 'EndDate', 'Name',
       'CurrentName', 'Active', 'Preview', 'BaseCost', 'PaymentFrequency',
       'PaymentFrequencyDerived_in_days', 'PaymentPeriod', 'SupplierID',
       'GlobalShowOnWhistleout', 'MobileProductKey', 'EditorsPickName'],
      dtype='object')

In [ ]:
product_df.shape

(659, 16)

# DimUTSCampaign Table

In [ ]:
campaign_df= pd.read_csv("/content/drive/Shareddrives/protik/iLab/Dataset/data/DimUTSCampaign.csv")

In [ ]:
campaign_df.head()

,CampaignKey,ID,StartDate,EndDate,Active,Preview,Name,Sequence,BusinessStartDate,BusinessEndDate,...,ExtraIncentive12mthTCOTotal,ExtraIncentiveOneOffTotal,PhonePcntIncentive1mthTCOTotal,PhonePcntIncentive6mthTCOTotal,PhonePcntIncentive12mthTCOTotal,PhonePcntIncentiveOneOffTotal,TabletPcntIncentive1mthTCOTotal,TabletPcntIncentive6mthTCOTotal,TabletPcntIncentive12mthTCOTotal,TabletPcntIncentiveOneOffTotal
0,20666,15412,2022-12-03 00:00:00.0000000,2023-02-28 00:00:00.0000000,1,0,$35 SIM Only,83,2022-12-03 00:00:00.0000000,2023-02-28 00:00:00.0000000,...,252000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,23436,15768,2023-01-13 00:00:00.0000000,2023-02-28 00:00:00.0000000,1,0,4G 100GB Mobile Plan,10,2023-01-13 00:00:00.0000000,2023-02-28 00:00:00.0000000,...,600000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,21546,15180,2022-12-01 00:00:00.0000000,2023-02-01 00:00:00.0000000,1,0,$12 Unlimited Plan,14,2022-12-01 00:00:00.0000000,2023-01-31 00:00:00.0000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,22097,15788,2023-01-11 00:00:00.0000000,2023-02-28 00:00:00.0000000,1,0,5G Ultimate Home Internet,28,2023-01-11 00:00:00.0000000,2023-02-28 00:00:00.0000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,14011,7360,2022-11-01 00:00:00.0000000,2023-02-28 00:00:00.0000000,1,0,$30 Pre-Paid Mobile,100,2022-11-01 00:00:00.0000000,2023-02-28 00:00:00.0000000,...,240000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
campaign_df.shape

(1578, 47)

In [ ]:
campaign_df.columns

Index(['CampaignKey', 'ID', 'StartDate', 'EndDate', 'Active', 'Preview',
       'Name', 'Sequence', 'BusinessStartDate', 'BusinessEndDate',
       'PromotionName', 'ProductAreaName', 'SupplierID', 'Exclusive',
       'PriceComment', 'OtherIncentive1mthTCOTotal',
       'OtherIncentive6mthTCOTotal', 'OtherIncentive12mthTCOTotal',
       'OtherIncentiveOneOffTotal', 'FixedIncentive1mthTCOTotal',
       'FixedIncentive6mthTCOTotal', 'FixedIncentive12mthTCOTotal',
       'FixedIncentiveOneOffTotal', 'PercentageIncentive1mthTCOTotal',
       'PercentageIncentive6mthTCOTotal', 'PercentageIncentive12mthTCOTotal',
       'PercentageIncentiveOneOffTotal', 'TabletIncentive1mthTCOTotal',
       'TabletIncentive6mthTCOTotal', 'TabletIncentive12mthTCOTotal',
       'TabletIncentiveOneOffTotal', 'PhoneIncentive1mthTCOTotal',
       'PhoneIncentive6mthTCOTotal', 'PhoneIncentive12mthTCOTotal',
       'PhoneIncentiveOneOffTotal', 'ExtraIncentive1mthTCOTotal',
       'ExtraIncentive6mthTCOTotal', 'Extra

In [ ]:
campaign_df = campaign_df.drop_duplicates()

In [ ]:
campaign_df['EndDate'] = pd.to_datetime(campaign_df['EndDate'])

campaign_df = campaign_df[campaign_df['EndDate']>='2022-12-01']

In [ ]:
campaign_df.shape

(1438, 47)

In [ ]:
campaign_df['BusinessEndDate'] = pd.to_datetime(campaign_df['BusinessEndDate'])

campaign_df = campaign_df[campaign_df['BusinessEndDate']>='2022-12-01']

In [ ]:
campaign_df.shape

(1316, 47)

In [ ]:
campaign_df = campaign_df[campaign_df['ProductAreaName'] == 'Mobile Phones']

In [ ]:
campaign_df.shape

(860, 47)

In [ ]:
# Define the substrings to search for in column names
substrings_to_search = ['key', 'comment', 'id', 'tablet']

# Get the column names that contain those word
matching_columns = [col for col in campaign_df.columns if any(sub in col.lower() for sub in substrings_to_search)]

# Print the matching column names
print("Columns containing 'key,' 'comment,', 'id' or 'tablet in their names:")
for col in matching_columns:
    print(col)

Columns containing 'key,' 'comment,', 'id' or 'tablet in their names:
CampaignKey
ID
SupplierID
PriceComment
TabletIncentive1mthTCOTotal
TabletIncentive6mthTCOTotal
TabletIncentive12mthTCOTotal
TabletIncentiveOneOffTotal
TabletPcntIncentive1mthTCOTotal
TabletPcntIncentive6mthTCOTotal
TabletPcntIncentive12mthTCOTotal
TabletPcntIncentiveOneOffTotal


In [ ]:
'''
CampaignKey = unique key for the campaign type
ID = unique identifier of an individual campaign
'''
columns_to_drop = [
    'ProductAreaName',
    'PriceComment',
    'TabletPcntIncentive1mthTCOTotal',
    'TabletPcntIncentive6mthTCOTotal',
    'TabletPcntIncentive12mthTCOTotal',
    'TabletPcntIncentiveOneOffTotal',
    'TabletIncentive1mthTCOTotal',
    'TabletIncentive6mthTCOTotal',
    'TabletIncentive12mthTCOTotal',
    'TabletIncentiveOneOffTotal'

]
# Drop the specified columns
campaign_df = campaign_df.drop(columns=columns_to_drop)

In [ ]:
campaign_df.rename(columns={'ID':'campaign_indv_ID'}, inplace=True)

In [ ]:
constant_columns = []

for column in campaign_df.columns:
    if campaign_df[column].nunique() == 1:
        constant_columns.append(column)

# Print the constant columns
print("Constant Columns:")
print(constant_columns)

Constant Columns:
['PhoneIncentiveOneOffTotal', 'PhonePcntIncentive1mthTCOTotal', 'PhonePcntIncentive6mthTCOTotal', 'PhonePcntIncentive12mthTCOTotal', 'PhonePcntIncentiveOneOffTotal']


In [ ]:
columns_to_drop = [
    'PhoneIncentiveOneOffTotal',
    'PhonePcntIncentive1mthTCOTotal',
    'PhonePcntIncentive6mthTCOTotal',
    'PhonePcntIncentive12mthTCOTotal',
    'PhonePcntIncentiveOneOffTotal'
]
# Drop the specified columns
campaign_df = campaign_df.drop(columns=columns_to_drop)

In [ ]:
campaign_df.shape

(860, 32)

In [ ]:
campaign_df.columns

Index(['CampaignKey', 'campaign_indv_ID', 'StartDate', 'EndDate', 'Active',
       'Preview', 'Name', 'Sequence', 'BusinessStartDate', 'BusinessEndDate',
       'PromotionName', 'SupplierID', 'Exclusive',
       'OtherIncentive1mthTCOTotal', 'OtherIncentive6mthTCOTotal',
       'OtherIncentive12mthTCOTotal', 'OtherIncentiveOneOffTotal',
       'FixedIncentive1mthTCOTotal', 'FixedIncentive6mthTCOTotal',
       'FixedIncentive12mthTCOTotal', 'FixedIncentiveOneOffTotal',
       'PercentageIncentive1mthTCOTotal', 'PercentageIncentive6mthTCOTotal',
       'PercentageIncentive12mthTCOTotal', 'PercentageIncentiveOneOffTotal',
       'PhoneIncentive1mthTCOTotal', 'PhoneIncentive6mthTCOTotal',
       'PhoneIncentive12mthTCOTotal', 'ExtraIncentive1mthTCOTotal',
       'ExtraIncentive6mthTCOTotal', 'ExtraIncentive12mthTCOTotal',
       'ExtraIncentiveOneOffTotal'],
      dtype='object')

In [ ]:
campaign_df['CampaignKey'].nunique()

860

# DimUTSSupplier Table

In [ ]:
supplier_df= pd.read_csv("/content/drive/Shareddrives/protik/iLab/Dataset/data/DimUTSSupplier.csv")

In [ ]:
supplier_df.head()

,SupplierKey,ID,StartDate,EndDate,Name,Active,ShortDescription,AwardsName
0,3786,377,2023-02-22 00:00:00.0000000,2023-02-28 00:00:00.0000000,Exetel,1,This telco launched in 2004 with a mission to ...,"Best Internet Promotion 2021, Best Home Wirele..."
1,3486,561,2023-02-22 00:00:00.0000000,2023-02-28 00:00:00.0000000,Moose Mobile,1,Moose is a low cost Australian service provide...,"Best SIM Only Provider Value Plans 2022, Best ..."
2,1174,279,2022-10-10 00:00:00.0000000,2023-02-28 00:00:00.0000000,iPrimus,1,Offering mobile & broadband services since 199...,NaN
3,1003,454,2022-10-06 00:00:00.0000000,2023-02-22 00:00:00.0000000,SpinTel,1,Started in 1996 they are an innovative and Mu...,"Best Home Wireless Provider 2021, Best Home Wi..."
4,1183,495,2022-10-11 00:00:00.0000000,2023-02-28 00:00:00.0000000,ALDI mobile,1,"Owned by the global retail chain, Aldi mobile ...",Best Prepaid Provider Value Plans 2020


In [ ]:
supplier_df.shape

(67, 8)

In [ ]:
supplier_df.columns

Index(['SupplierKey', 'ID', 'StartDate', 'EndDate', 'Name', 'Active',
       'ShortDescription', 'AwardsName'],
      dtype='object')

In [ ]:
supplier_df['EndDate'] = pd.to_datetime(supplier_df['EndDate'])

supplier_df = supplier_df[supplier_df['EndDate']>='2022-12-01']

In [ ]:
supplier_df.shape

(66, 8)

In [ ]:
# Calculate the total number of missing values per column
missing_values = supplier_df.isnull().sum()

# Calculate the percentage of missing values per column
percentage_missing = (missing_values / len(supplier_df)) * 100

# Create a DataFrame to display the results
missing_data_info = pd.DataFrame({
    'Column Name': missing_values.index,
    'Total Missing Values': missing_values.values,
    'Percentage Missing': percentage_missing.values
})

# Sort the DataFrame by percentage of missing values (descending)
missing_data_info = missing_data_info.sort_values(by='Percentage Missing', ascending=False)

# Print the missing data information
missing_data_info

,Column Name,Total Missing Values,Percentage Missing
7,AwardsName,31,46.969697
6,ShortDescription,8,12.121212
4,Name,1,1.515152
0,SupplierKey,0,0.000000
1,ID,0,0.000000
2,StartDate,0,0.000000
3,EndDate,0,0.000000
5,Active,0,0.000000


In [ ]:
columns_to_drop = [
    'AwardsName',
    'ShortDescription'
]
# Drop the specified columns
supplier_df = supplier_df.drop(columns=columns_to_drop)

In [ ]:
supplier_df = supplier_df.drop_duplicates()

In [ ]:
supplier_df.shape

(66, 6)

In [ ]:
supplier_df.rename(columns={'ID':'supplier_indv_ID'}, inplace=True)

In [ ]:
supplier_df.columns

Index(['SupplierKey', 'supplier_indv_ID', 'StartDate', 'EndDate', 'Name',
       'Active'],
      dtype='object')

# DimUTSMobileProduct Table

In [ ]:
mobileproduct_df= pd.read_csv("/content/drive/Shareddrives/protik/iLab/Dataset/data/DimUTSMobileProduct.csv")

In [ ]:
mobileproduct_df.head()

,MobileProductKey,ProductID,StartDate,EndDate,Deleted,DomesticFlagFall,DomesticCallRate,DomesticSmsRate,DomesticMmsRate,DomesticVideoFlagFall,...,IncludedValueContainsMms,IncludedValueContainsVideo,IncludedDataComment,UnlimitedDataComment,MobilePlanTypeName,MobilePlanTypeNameDerived,is5GNetwork,MobilePhonePlan,MobilePhoneNetworkName,MobilePhoneNetworkID
0,2753,13057,2022-11-04 00:00:00.0000000,2023-02-28 00:00:00.0000000,0,0.0,0.0,0.0,0.0,0.0,...,1,0,within Australia,NaN,Postpaid,Postpaid,1,1,Optus,4
1,2686,8960,2022-10-31 00:00:00.0000000,2023-02-28 00:00:00.0000000,0,0.0,0.0,0.0,0.0,0.0,...,1,0,NaN,NaN,Postpaid,Postpaid,0,0,Optus,4
2,2226,12497,2022-11-27 00:00:00.0000000,2023-02-28 00:00:00.0000000,0,0.0,0.0,0.0,0.0,0.0,...,1,1,at Max Speed,NaN,Postpaid,Postpaid,1,1,Vodafone,6
3,3239,11852,2022-10-23 00:00:00.0000000,2023-02-28 00:00:00.0000000,0,0.0,0.0,0.0,0.0,0.0,...,1,1,for use in Australia,NaN,Postpaid,Postpaid,1,1,Telstra,2
4,2209,12325,2022-12-01 00:00:00.0000000,2023-02-28 00:00:00.0000000,0,0.0,0.0,0.0,0.0,0.0,...,1,0,over 365 days,NaN,Prepaid,Prepaid,0,0,Vodafone,6


In [ ]:
mobileproduct_df.shape

(721, 33)

In [ ]:
mobileproduct_df.columns

Index(['MobileProductKey', 'ProductID', 'StartDate', 'EndDate', 'Deleted',
       'DomesticFlagFall', 'DomesticCallRate', 'DomesticSmsRate',
       'DomesticMmsRate', 'DomesticVideoFlagFall', 'DomesticVideoCallRate',
       'IncludedValue', 'IncludedData', 'IncludedDataDescriptionMB',
       'IncludedDataDescriptionGB', 'DataRate', 'IncludedMessages',
       'IncludedValueComment', 'IncludedMessagesComment',
       'UnlimitedValueComment', 'DataRateComment',
       'DataRateOverrideDescription', 'IncludedValueContainsSms',
       'IncludedValueContainsMms', 'IncludedValueContainsVideo',
       'IncludedDataComment', 'UnlimitedDataComment', 'MobilePlanTypeName',
       'MobilePlanTypeNameDerived', 'is5GNetwork', 'MobilePhonePlan',
       'MobilePhoneNetworkName', 'MobilePhoneNetworkID'],
      dtype='object')

In [ ]:
mobileproduct_df['EndDate'] = pd.to_datetime(mobileproduct_df['EndDate'])

mobileproduct_df = mobileproduct_df[mobileproduct_df['EndDate']>='2022-12-01']

In [ ]:
mobileproduct_df.shape

(706, 33)

In [ ]:
mobileproduct_df = mobileproduct_df.drop_duplicates()

In [ ]:
mobileproduct_df.shape

(315, 33)

In [ ]:
# Calculate the total number of missing values per column
missing_values = mobileproduct_df.isnull().sum()

# Calculate the percentage of missing values per column
percentage_missing = (missing_values / len(mobileproduct_df)) * 100

# Create a DataFrame to display the results
missing_data_info = pd.DataFrame({
    'Column Name': missing_values.index,
    'Total Missing Values': missing_values.values,
    'Percentage Missing': percentage_missing.values
})

# Sort the DataFrame by percentage of missing values (descending)
missing_data_info = missing_data_info.sort_values(by='Percentage Missing', ascending=False)

# Print the missing data information
missing_data_info

,Column Name,Total Missing Values,Percentage Missing
26,UnlimitedDataComment,315,100.000000
20,DataRateComment,306,97.142857
19,UnlimitedValueComment,302,95.873016
25,IncludedDataComment,175,55.555556
21,DataRateOverrideDescription,39,12.380952
18,IncludedMessagesComment,13,4.126984
17,IncludedValueComment,2,0.634921
0,MobileProductKey,0,0.000000
31,MobilePhoneNetworkName,0,0.000000
30,MobilePhonePlan,0,0.000000


In [ ]:
# Filter for columns with more than 50% missing values
columns_with_more_than_50_percent_missing = missing_data_info[missing_data_info['Percentage Missing'] > 50]

# Print the filtered information
print("Columns with more than 50% missing values:")
print(columns_with_more_than_50_percent_missing)

Columns with more than 50% missing values:
              Column Name  Total Missing Values  Percentage Missing
26   UnlimitedDataComment                   315          100.000000
20        DataRateComment                   306           97.142857
19  UnlimitedValueComment                   302           95.873016
25    IncludedDataComment                   175           55.555556


In [ ]:
# List of columns to drop which has description in it
columns_to_drop = [
    'UnlimitedDataComment',
    'DataRateComment',
    'UnlimitedValueComment',
    'IncludedDataComment'
]

# Drop the specified columns
mobileproduct_df = mobileproduct_df.drop(columns=columns_to_drop)

In [ ]:
# Define the substrings to search for in column names
substrings_to_search = ['key', 'comment', 'id','description']

# Get the column names that contain those word
matching_columns = [col for col in mobileproduct_df.columns if any(sub in col.lower() for sub in substrings_to_search)]

# Print the matching column names
print("Columns containing 'key,' 'comment,', 'id' or 'description in their names:")
for col in matching_columns:
    print(col)

Columns containing 'key,' 'comment,', 'id' or 'description in their names:
MobileProductKey
ProductID
DomesticVideoFlagFall
DomesticVideoCallRate
IncludedDataDescriptionMB
IncludedDataDescriptionGB
IncludedValueComment
IncludedMessagesComment
DataRateOverrideDescription
IncludedValueContainsVideo
MobilePhoneNetworkID


In [ ]:
# List of columns to drop which has description in it
columns_to_drop = [
    'IncludedValueComment',
    'IncludedMessagesComment',
    'DataRateOverrideDescription',
]

# Drop the specified columns
mobileproduct_df = mobileproduct_df.drop(columns=columns_to_drop)

In [ ]:
constant_columns = []

for column in mobileproduct_df.columns:
    if mobileproduct_df[column].nunique() == 1:
        constant_columns.append(column)

# Print the constant columns
print("Constant Columns:")
print(constant_columns)

Constant Columns:
['Deleted']


In [ ]:
# List of columns to drop which has description in it
columns_to_drop = [
    'Deleted',
]

# Drop the specified columns
mobileproduct_df = mobileproduct_df.drop(columns=columns_to_drop)

In [ ]:
mobileproduct_df.shape

(315, 25)

In [ ]:
mobileproduct_df.columns

Index(['MobileProductKey', 'ProductID', 'StartDate', 'EndDate',
       'DomesticFlagFall', 'DomesticCallRate', 'DomesticSmsRate',
       'DomesticMmsRate', 'DomesticVideoFlagFall', 'DomesticVideoCallRate',
       'IncludedValue', 'IncludedData', 'IncludedDataDescriptionMB',
       'IncludedDataDescriptionGB', 'DataRate', 'IncludedMessages',
       'IncludedValueContainsSms', 'IncludedValueContainsMms',
       'IncludedValueContainsVideo', 'MobilePlanTypeName',
       'MobilePlanTypeNameDerived', 'is5GNetwork', 'MobilePhonePlan',
       'MobilePhoneNetworkName', 'MobilePhoneNetworkID'],
      dtype='object')

In [ ]:
mobileproduct_df.rename(columns={'IncludedData':'IncludedDataMB'}, inplace=True)

In [ ]:
mobileproduct_df.columns

Index(['MobileProductKey', 'ProductID', 'StartDate', 'EndDate',
       'DomesticFlagFall', 'DomesticCallRate', 'DomesticSmsRate',
       'DomesticMmsRate', 'DomesticVideoFlagFall', 'DomesticVideoCallRate',
       'IncludedValue', 'IncludedDataMB', 'IncludedDataDescriptionMB',
       'IncludedDataDescriptionGB', 'DataRate', 'IncludedMessages',
       'IncludedValueContainsSms', 'IncludedValueContainsMms',
       'IncludedValueContainsVideo', 'MobilePlanTypeName',
       'MobilePlanTypeNameDerived', 'is5GNetwork', 'MobilePhonePlan',
       'MobilePhoneNetworkName', 'MobilePhoneNetworkID'],
      dtype='object')

In [ ]:
# MB GB

# Joining Data

In [ ]:
fact_df.columns

Index(['session_ID', 'ViewedOnTime', 'ViewedOnDate', 'ProductKey',
       'CampaignKey', 'UserIdKey', 'SupplierKey', 'PageCount', 'IsAd',
       'IsTransaction', 'IsProduct', 'AffiliateName', 'DeviceType',
       'ContractLengthMonths', 'ContractLengthDesc', 'Source', 'Medium',
       'Campaign', 'WOHeadlinePlanFixedDiscount', 'WOHeadlinePhoneDiscount',
       'WOHeadlineExtraDataDiscount', 'OneOffFeeTotal', 'Phonemonthlycost'],
      dtype='object')

In [ ]:
product_df.columns

Index(['ProductKey', 'product_indv_ID', 'StartDate', 'EndDate', 'Name',
       'CurrentName', 'Active', 'Preview', 'BaseCost', 'PaymentFrequency',
       'PaymentFrequencyDerived_in_days', 'PaymentPeriod', 'SupplierID',
       'GlobalShowOnWhistleout', 'MobileProductKey', 'EditorsPickName'],
      dtype='object')

In [ ]:
campaign_df.columns

Index(['CampaignKey', 'campaign_indv_ID', 'StartDate', 'EndDate', 'Active',
       'Preview', 'Name', 'Sequence', 'BusinessStartDate', 'BusinessEndDate',
       'PromotionName', 'SupplierID', 'Exclusive',
       'OtherIncentive1mthTCOTotal', 'OtherIncentive6mthTCOTotal',
       'OtherIncentive12mthTCOTotal', 'OtherIncentiveOneOffTotal',
       'FixedIncentive1mthTCOTotal', 'FixedIncentive6mthTCOTotal',
       'FixedIncentive12mthTCOTotal', 'FixedIncentiveOneOffTotal',
       'PercentageIncentive1mthTCOTotal', 'PercentageIncentive6mthTCOTotal',
       'PercentageIncentive12mthTCOTotal', 'PercentageIncentiveOneOffTotal',
       'PhoneIncentive1mthTCOTotal', 'PhoneIncentive6mthTCOTotal',
       'PhoneIncentive12mthTCOTotal', 'ExtraIncentive1mthTCOTotal',
       'ExtraIncentive6mthTCOTotal', 'ExtraIncentive12mthTCOTotal',
       'ExtraIncentiveOneOffTotal'],
      dtype='object')

In [ ]:
supplier_df.columns

Index(['SupplierKey', 'supplier_indv_ID', 'StartDate', 'EndDate', 'Name',
       'Active'],
      dtype='object')

In [ ]:
mobileproduct_df.columns

Index(['MobileProductKey', 'ProductID', 'StartDate', 'EndDate',
       'DomesticFlagFall', 'DomesticCallRate', 'DomesticSmsRate',
       'DomesticMmsRate', 'DomesticVideoFlagFall', 'DomesticVideoCallRate',
       'IncludedValue', 'IncludedDataMB', 'IncludedDataDescriptionMB',
       'IncludedDataDescriptionGB', 'DataRate', 'IncludedMessages',
       'IncludedValueContainsSms', 'IncludedValueContainsMms',
       'IncludedValueContainsVideo', 'MobilePlanTypeName',
       'MobilePlanTypeNameDerived', 'is5GNetwork', 'MobilePhonePlan',
       'MobilePhoneNetworkName', 'MobilePhoneNetworkID'],
      dtype='object')

In [ ]:
# # Perform LEFT JOINs using Pandas merge()
# merged_data_inner = fact_df.merge(product_df, left_on='ProductKey', right_on= 'ProductKey', how='inner', suffixes=('', '_product_table')) \
#                         .merge(campaign_df, left_on='CampaignKey', right_on= 'CampaignKey', how='inner', suffixes=('', '_campaign_table')) \
#                         .merge(supplier_df, left_on='SupplierKey', right_on= 'SupplierKey', how='inner', suffixes=('', '_supplier_table'))

# merged_data_inner = merged_data_inner.merge(mobileproduct_df, left_on='MobileProductKey', right_on='MobileProductKey', how='inner', suffixes=('', '_mobileproduct_table'))


In [ ]:
# Perform LEFT JOINs using Pandas merge()
merged_data = fact_df.merge(product_df, left_on='ProductKey', right_on= 'ProductKey', how='left', suffixes=('', '_product_table')) \
                        .merge(campaign_df, left_on='CampaignKey', right_on= 'CampaignKey', how='left', suffixes=('', '_campaign_table')) \
                        .merge(supplier_df, left_on='SupplierKey', right_on= 'SupplierKey', how='left', suffixes=('', '_supplier_table'))

merged_data = merged_data.merge(mobileproduct_df, left_on='MobileProductKey', right_on='MobileProductKey', how='left', suffixes=('', '_mobileproduct_table'))


In [ ]:
merged_data.shape

(483275, 98)

In [ ]:
merged_data.IsTransaction.sum()

149104

In [ ]:
# merged_data_inner.shape

(122087, 98)

In [ ]:
# merged_data_inner.IsTransaction.sum()

118231

In [ ]:
merged_data['EditorsPickName'].nunique()

2

In [ ]:
df = merged_data

In [ ]:
# column names
column_names = df.columns.tolist()
for i, column_name in enumerate(column_names):
    print(f"Columns {i+1}: {column_name}")

Columns 1: session_ID
Columns 2: ViewedOnTime
Columns 3: ViewedOnDate
Columns 4: ProductKey
Columns 5: CampaignKey
Columns 6: UserIdKey
Columns 7: SupplierKey
Columns 8: PageCount
Columns 9: IsAd
Columns 10: IsTransaction
Columns 11: IsProduct
Columns 12: AffiliateName
Columns 13: DeviceType
Columns 14: ContractLengthMonths
Columns 15: ContractLengthDesc
Columns 16: Source
Columns 17: Medium
Columns 18: Campaign
Columns 19: WOHeadlinePlanFixedDiscount
Columns 20: WOHeadlinePhoneDiscount
Columns 21: WOHeadlineExtraDataDiscount
Columns 22: OneOffFeeTotal
Columns 23: Phonemonthlycost
Columns 24: product_indv_ID
Columns 25: StartDate
Columns 26: EndDate
Columns 27: Name
Columns 28: CurrentName
Columns 29: Active
Columns 30: Preview
Columns 31: BaseCost
Columns 32: PaymentFrequency
Columns 33: PaymentFrequencyDerived_in_days
Columns 34: PaymentPeriod
Columns 35: SupplierID
Columns 36: GlobalShowOnWhistleout
Columns 37: MobileProductKey
Columns 38: EditorsPickName
Columns 39: campaign_indv_I

In [ ]:
# Calculate the total number of missing values per column
missing_values = df.isnull().sum()

# Calculate the percentage of missing values per column
percentage_missing = (missing_values / len(df)) * 100

# Create a DataFrame to display the results
missing_data_info = pd.DataFrame({
    'Column Name': missing_values.index,
    'Total Missing Values': missing_values.values,
    'Percentage Missing': percentage_missing.values
})

# Sort the DataFrame by percentage of missing values (descending)
missing_data_info = missing_data_info.sort_values(by='Percentage Missing', ascending=False)

# Print the missing data information
missing_data_info

,Column Name,Total Missing Values,Percentage Missing
49,Exclusive,361188,74.737572
40,EndDate_campaign_table,361188,74.737572
64,PhoneIncentive12mthTCOTotal,361188,74.737572
63,PhoneIncentive6mthTCOTotal,361188,74.737572
62,PhoneIncentive1mthTCOTotal,361188,74.737572
...,...,...,...
73,Active_supplier_table,0,0.000000
71,EndDate_supplier_table,0,0.000000
70,StartDate_supplier_table,0,0.000000
69,supplier_indv_ID,0,0.000000


In [ ]:
constant_columns = []

for column in df.columns:
    if df[column].nunique() == 1:
        constant_columns.append(column)

# Print the constant columns
print("Constant Columns:")
print(constant_columns)

Constant Columns:
[]


In [ ]:
columns_to_drop =[
    'ContractLengthMonths',
    'ContractLengthDesc',
    'Active_supplier_table', # will show all of them are active right now
    'IncludedValue',
    'IncludedMessages',
    'IncludedValueContainsSms']

df = df.drop(columns=columns_to_drop)

In [ ]:
df.shape

(483275, 92)

In [ ]:
df.columns

Index(['session_ID', 'ViewedOnTime', 'ViewedOnDate', 'ProductKey',
       'CampaignKey', 'UserIdKey', 'SupplierKey', 'PageCount', 'IsAd',
       'IsTransaction', 'IsProduct', 'AffiliateName', 'DeviceType', 'Source',
       'Medium', 'Campaign', 'WOHeadlinePlanFixedDiscount',
       'WOHeadlinePhoneDiscount', 'WOHeadlineExtraDataDiscount',
       'OneOffFeeTotal', 'Phonemonthlycost', 'product_indv_ID', 'StartDate',
       'EndDate', 'Name', 'CurrentName', 'Active', 'Preview', 'BaseCost',
       'PaymentFrequency', 'PaymentFrequencyDerived_in_days', 'PaymentPeriod',
       'SupplierID', 'GlobalShowOnWhistleout', 'MobileProductKey',
       'EditorsPickName', 'campaign_indv_ID', 'StartDate_campaign_table',
       'EndDate_campaign_table', 'Active_campaign_table',
       'Preview_campaign_table', 'Name_campaign_table', 'Sequence',
       'BusinessStartDate', 'BusinessEndDate', 'PromotionName',
       'SupplierID_campaign_table', 'Exclusive', 'OtherIncentive1mthTCOTotal',
       'OtherIncent

# Downloading merged data

In [ ]:
# df.to_csv('/content/drive/Shareddrives/protik/iLab/Dataset/clean/finalbq.csv', index=False)

In [ ]:
df.to_csv('/content/drive/Shareddrives/protik/iLab/Dataset/clean/leftjoin.csv', index=False)

# Analysis

In [ ]:
# protik said that day about some bar chart